## BLOQUE 0 — Infraestructura, Rutas Maestras y Entorno Hardcore (v5.4)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v5.4 (Resiliencia de Directorios)
Sello: 2026-01-29 [jue] 10:20

🎯 Propósito
Establecer la infraestructura de producción. Este bloque inicializa el entorno, monta Google Drive y valida la existencia de los activos maestros. Implementa una lógica de persistencia que asegura la creación del directorio de salida antes de cualquier operación de escritura.

📥 Entradas
- Excel Maestro: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx
- Logo Circular: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png

📤 Salidas
- Variables globales EXCEL_PATH, LOGO_PATH y OUTPUT_DIR validadas.
- Directorio de salida físico creado en Google Drive.

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Validación de Existencia: El script verifica físicamente los archivos en Drive.
- [ ] Persistencia Atómica: Generación de RUN_ID único para evitar colisiones.
- [ ] Auto-Recuperación: El bloque puede ser ejecutado múltiples veces sin romper la sesión.

In [36]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    10:20
# Versión: v5.4
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: BLOQUE 0 - Inicialización de Infraestructura y Rutas Maestras
# -----------------------------------------------------------------------------

import os
import sys
import subprocess
from datetime import datetime

def initialize_production_environment():
    """Inicializa entorno, monta Drive y valida rutas maestras."""
    print("📡 [SISTEMA] Desplegando Arquitectura Hardcore v5.4...")

    # 1. Instalación de dependencias
    libs = ["pyarrow", "python-pptx", "openpyxl", "pandas", "matplotlib", "fpdf2", "numpy"]
    for lib in libs:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", lib])

    import pandas as pd
    from google.colab import drive

    # 2. Montaje de Drive
    if not os.path.exists('/content/drive'):
        print("📡 [DRIVE] Solicitando acceso a la unidad...")
        drive.mount('/content/drive')

    # 3. RUTAS MAESTRAS DEFINIDAS
    EXCEL_PATH_FINAL = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx"
    LOGO_PATH_FINAL = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png"

    # 4. GESTIÓN DE SALIDAS (Trazabilidad)
    TIMESTAMP = datetime.now().strftime("%Y-%m-%d_%H%M")
    RUN_ID = f"CRSN_PROD_v5.4_{TIMESTAMP}"
    BASE_OUT = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025"
    OUTPUT_DIR_FINAL = os.path.join(BASE_OUT, RUN_ID)

    # CREACIÓN FÍSICA DEL DIRECTORIO
    os.makedirs(OUTPUT_DIR_FINAL, exist_ok=True)
    print(f"✅ [SISTEMA] Directorio de salida verificado: {OUTPUT_DIR_FINAL}")

    # 5. VERIFICACIÓN DE ACTIVOS
    activos = {"Excel": EXCEL_PATH_FINAL, "Logo": LOGO_PATH_FINAL}
    for nombre, path in activos.items():
        if os.path.exists(path):
            print(f"✅ [EXISTE] {nombre} en: {path}")
        else:
            print(f"❌ [ERROR] {nombre} NO hallado en {path}")
            raise FileNotFoundError(f"Falta activo crítico: {path}")

    return EXCEL_PATH_FINAL, LOGO_PATH_FINAL, OUTPUT_DIR_FINAL

# EJECUCIÓN GLOBAL
try:
    EXCEL_PATH, LOGO_PATH, OUTPUT_DIR = initialize_production_environment()
    print(f"\n✅ [BLOQUE 0] FINALIZADO. Listo para procesamiento.")
except Exception as e:
    print(f"❌ [CRÍTICO] Fallo en la inicialización: {e}")

📡 [SISTEMA] Desplegando Arquitectura Hardcore v5.4...
✅ [SISTEMA] Directorio de salida verificado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/CRSN_PROD_v5.4_2026-01-29_1658
✅ [EXISTE] Excel en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx
✅ [EXISTE] Logo en: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png

✅ [BLOQUE 0] FINALIZADO. Listo para procesamiento.


## BLOQUE 1 — Entregable Maestro Diezmos (v6.9)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v6.9 (Gold Standard - Índice 400 Verificado)
Sello: 2026-01-29 [jue] 12:20:00

🎯 Propósito
Generar la pieza visual final de Diezmos. Se confirma el Índice 400 para 2025 basado en la sumatoria estricta del archivo 'vf'. El diseño es 16:9 panorámico con estética Dark Minimalist.

📥 Entradas
- Excel Maestro Final: vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx
- Logo Dark: crsn-logo-dark-circulo.png

📤 Salidas
- [YYYYMMDD_HHMMSS]_DIEZMOS_DARK_16x9_v6.9.pptx (Archivo para PowerPoint).
- df_diezmos_final_400.csv (Dataset de auditoría).

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Validación 400: Sumatoria anual 2025 ($6.1B) / 2015 ($1.5B) = 4.0.
- [ ] Safe Zone: Margen del 5% aplicado para visualización perfecta.
- [ ] Nomenclatura PJLA: ID único con segundos para control de versiones.

In [43]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    12:20:00
# Versión: v6.9 (Gold Standard)
# Propósito: Generación de Slide DARK 16:9 con Índice 400
# -----------------------------------------------------------------------------

import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from datetime import datetime

# RUTAS MAESTRAS
EXCEL_PATH = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx"
LOGO_DARK = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png"
OUTPUT_DIR = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/"

def generar_entrega_final_v69(excel, logo, out):
    # 1. Extracción de Datos
    xl = pd.ExcelFile(excel)
    results = []
    for yr in [str(y) for y in range(2015, 2026)]:
        df_sheet = pd.read_excel(xl, sheet_name=yr, header=None)
        mask = df_sheet[0].astype(str).str.strip().str.upper() == "DIEZMOS"
        row = df_sheet[mask]
        if not row.empty:
            results.append({"AÑO": int(yr), "VALOR": pd.to_numeric(row.iloc[0, 1:13], errors='coerce').sum()})

    df = pd.DataFrame(results)
    base_15 = df.loc[df['AÑO'] == 2015, 'VALOR'].values[0]
    df['INDICE'] = ((df['VALOR'] / base_15) * 100).round(0).astype(int)

    # 2. Nomenclatura PJLA con Segundos
    pjla_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{pjla_id}_DIEZMOS_DARK_16x9_v6.9.pptx"
    save_path = os.path.join(out, filename)

    # 3. Presentación 16:9
    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.33), Inches(7.5)
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    slide.background.fill.solid()
    slide.background.fill.fore_color.rgb = RGBColor(10, 10, 20)

    if os.path.exists(logo):
        slide.shapes.add_picture(logo, Inches(0.6), Inches(0.5), height=Inches(1.3))

    # Títulos
    t_box = slide.shapes.add_textbox(Inches(2.5), Inches(0.4), Inches(10), Inches(1))
    p = t_box.text_frame.paragraphs[0]
    p.text = "DIEZMOS 2015-2025"
    p.font.bold, p.font.size, p.font.color.rgb = True, Pt(60), RGBColor(255, 255, 255)

    s_box = slide.shapes.add_textbox(Inches(2.5), Inches(1.35), Inches(4), Inches(0.5))
    sp = s_box.text_frame.paragraphs[0]
    sp.text = "2015=100"
    sp.font.size, sp.font.color.rgb = Pt(10), RGBColor(255, 255, 255)

    # Gráfico (Barras 95%)
    plt.figure(figsize=(14, 6), facecolor='#0A0A14')
    ax = plt.axes()
    ax.set_facecolor('#0A0A14')
    bars = plt.bar(df['AÑO'].astype(str), df['INDICE'], color='#1A73E8', width=0.95)

    plt.xticks(color='white', fontsize=18, fontweight='black')
    plt.yticks(color='white', fontsize=12)
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')

    for bar in bars:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{int(bar.get_height())}',
                 ha='center', va='bottom', color='white', fontweight='black', fontsize=20)

    plt.tight_layout(pad=3.5)
    img_tmp = f"chart_{pjla_id}.png"
    plt.savefig(img_tmp, dpi=300, bbox_inches='tight', facecolor='#0A0A14')
    plt.close()

    slide.shapes.add_picture(img_tmp, Inches(0.65), Inches(2.2), width=Inches(12))
    prs.save(save_path)
    os.remove(img_tmp)
    return save_path

# Ejecución
try:
    pptx_file = generar_entrega_final_v69(EXCEL_PATH, LOGO_DARK, OUTPUT_DIR)
    print(f"✅ Entregable Generado: {pptx_file}")
    from

SyntaxError: invalid syntax (ipython-input-3998946125.py, line 92)

## BLOQUE 3 — Renderizado DARK 16:9 con Blindaje de Márgenes (v6.2)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v6.2 (Control de Márgenes y Maximización de Fuente)
Sello: 2026-01-29 [jue] 11:15

🎯 Propósito
Corregir el recorte visual de los años y maximizar la legibilidad. Se establece un margen de seguridad del 5% en todos los bordes. Se amplía el tamaño de los años (eje X) y de las etiquetas de datos para que coincidan con el ancho de las columnas.

📥 Entradas
- df_diezmos_master: Serie histórica 2015-2025.
- LOGO_DARK: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png

📤 Salidas
- [PJLA_TIMESTAMP]_DIEZMOS_DARK_v6.2_FINAL.pptx

🛡️ Checks de Seguridad (Anti-Alucinación)
- [ ] Safe Zone Check: Margen de 0.5" en los 4 costados del slide.
- [ ] Escala de Texto: Años y datos en Bold con tamaño optimizado para el ancho de barra.
- [ ] Sin Recortes: Uso de plt.subplots_adjust para evitar el corte de labels inferiores.

In [38]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    11:15
# Versión: v6.2
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: BLOQUE 3 - Slide DARK 16:9 Sin Recortes y Máximo Resaltado
# -----------------------------------------------------------------------------

import os
import pandas as pd
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from datetime import datetime

# CONFIGURACIÓN DE RUTAS MAESTRAS
OUTPUT_FOLDER = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/"
LOGO_DARK = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png"

def generar_pptx_perfecto_v62(df, logo_path, out_dir):
    """Genera slide 16:9 con márgenes de seguridad y fuentes maximizadas."""

    # 1. NOMENCLATURA PJLA
    now = datetime.now()
    prefix = now.strftime("%Y%m%d_%H%M")
    filename = f"{prefix}_DIEZMOS_DARK_v6.2_FINAL.pptx"
    full_save_path = os.path.join(out_dir, filename)

    # 2. SETUP SLIDE 16:9 CON MÁRGENES (5%)
    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.33), Inches(7.5)
    slide = prs.slides.add_slide(prs.slide_layouts[6])

    # Fondo Dark Profundo
    background = slide.background
    background.fill.solid()
    background.fill.fore_color.rgb = RGBColor(10, 10, 20)

    # 3. LOGO DARK (Escala Áurea)
    if os.path.exists(logo_path):
        slide.shapes.add_picture(logo_path, Inches(0.6), Inches(0.5), height=Inches(1.2))

    # 4. TÍTULOS (Subtítulo 17% y pegado)
    t_size = 60
    s_size = int(t_size * 0.17)

    t_box = slide.shapes.add_textbox(Inches(2.4), Inches(0.4), Inches(10), Inches(1))
    p = t_box.text_frame.paragraphs[0]
    p.text = "DIEZMOS 2015-2025"
    p.font.bold, p.font.size, p.font.color.rgb = True, Pt(t_size), RGBColor(255, 255, 255)

    s_box = slide.shapes.add_textbox(Inches(2.4), Inches(1.3), Inches(4), Inches(0.5))
    sp = s_box.text_frame.paragraphs[0]
    sp.text = "2015=100"
    sp.font.size, sp.font.color.rgb = Pt(s_size), RGBColor(255, 255, 255)

    # 5. GRÁFICO MAXIMIZADO (Sin recortes)
    # Aumentamos DPI para nitidez de texto
    plt.figure(figsize=(14, 6), facecolor='#0A0A14')
    ax = plt.axes()
    ax.set_facecolor('#0A0A14')

    # width=0.95 para cubrir el ancho de la columna
    bars = plt.bar(df['AÑO'].astype(str), df['INDICE'], color='#1A73E8', alpha=0.9, width=0.95)

    # Ajuste de Ejes para evitar recortes (Margen 5%)
    plt.xticks(color='white', fontsize=16, fontweight='black') # Años gigantes y resaltados
    plt.yticks(color='white', fontsize=12)
    ax.spines['bottom'].set_color('white')
    ax.spines['left'].set_color('white')
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    plt.grid(axis='y', color='white', linestyle='--', alpha=0.05)

    # Etiquetas de Datos (Data Labels) maximizadas al ancho de columna
    for bar in bars:
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 5, f'{int(bar.get_height())}',
                 ha='center', va='bottom', color='white', fontweight='black', fontsize=18)

    # AJUSTE CRÍTICO: Previene el recorte de los años abajo
    plt.tight_layout(pad=3.0)

    temp_img = "final_slide_v62.png"
    plt.savefig(temp_img, dpi=300, bbox_inches='tight', facecolor='#0A0A14')
    plt.close()

    # Inserción Central con margen de seguridad del 5%
    # Slide de 13.33", imagen de 12" centrada
    slide.shapes.add_picture(temp_img, Inches(0.65), Inches(2.1), width=Inches(12))

    # 6. GUARDADO
    if not os.path.exists(out_dir): os.makedirs(out_dir)
    prs.save(full_save_path)
    if os.path.exists(temp_img): os.remove(temp_img)
    return full_save_path

# EJECUCIÓN
try:
    final_pptx = generar_pptx_perfecto_v62(df_diezmos_master, LOGO_DARK, OUTPUT_FOLDER)
    print(f"✅ [ÉXITO] Entregable v6.2 generado y blindado: {final_pptx}")
    from google.colab import files
    files.download(final_pptx)
except Exception as e:
    print(f"❌ [ERROR] Fallo en Bloque 3: {e}")

✅ [ÉXITO] Entregable v6.2 generado y blindado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260129_1658_DIEZMOS_DARK_v6.2_FINAL.pptx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## ESTÁNDARES DE PRODUCCIÓN — PROTOCOLO PJLA v6.2
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)

🎯 Reglas de Oro para Slides:
1. Formato: Siempre Panorámico 16:9 (13.33" x 7.5").
2. Estética: Dark Mode (Fondo Navy #0A0A14, Barras Azul Neón #1A73E8).
3. Márgenes: Safe Zone del 5% en todos los bordes (Sin recortes).
4. Barras: Ancho del 95% de la columna (Efecto bloque sólido).
5. Tipografía: Años y Data Labels en 'Black' (Extra Bold) maximizados al ancho de la barra.
6. Títulos: Principal (100% size) / Subtítulo (17% size) pegado inmediatamente abajo.
7. Logo: Versión Dark, Proporción Áurea, margen superior izquierdo.
8. Nomenclatura: [YYYYMMDD_HHMM]_Nombre_Slide_v6.2.pptx

In [39]:
# -----------------------------------------------------------------------------
# Protocolo de Sincronización GitHub - PJLA v1.1
# -----------------------------------------------------------------------------

import os

# CONFIGURACIÓN (Usa tus datos registrados)
GITHUB_USER = "tu_usuario"
GITHUB_TOKEN = "tu_token"
REPO_NAME = "crsn-slides-v5-hardcore"

# 1. Copiar activos desde Drive/Local al entorno de Git
# Aseguramos que el logo dark esté incluido
os.system("cp /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png .")
os.system("cp /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-circulo.png .")

# 2. Protocolo de Actualización
os.system("git add .")
os.system('git commit -m "Update v6.2: Implementación Gold Standard, Logo Dark y Formato 16:9"')

# 3. Empuje a Main (Push)
os.system("git push origin main")

print(f"🚀 [GITHUB] Repositorio actualizado con la versión v6.2")

🚀 [GITHUB] Repositorio actualizado con la versión v6.2


## BLOQUE 5 — Mapa de Calor Mensual 2015-2025 (v7.0)
Lead Data Scientist: Pedro J. Lancheros (Peter Boat)
Proyecto: CRSN Slides (3J / VitaNova)
Versión: v7.0 (Heatmap Gold Standard)
Sello: 2026-01-29 [jue] 12:35:00

🎯 Propósito
Visualizar la intensidad de la recaudación mensual de Diezmos a lo largo de una década en un solo slide. Permite identificar estacionalidad y picos históricos (como el récord de 2025) de forma inmediata.

📥 Entradas
- Excel Maestro Final: vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx

📤 Salidas
- [PJLA_ID]_DIEZMOS_HEATMAP_v7.0.pptx (Formato 16:9).
- [PJLA_ID]_DATA_HEATMAP_v7.0.csv (Matriz de datos para auditoría).

🛡️ Checks de Seguridad
- [ ] Integridad de Matriz: 11 años x 12 meses (132 puntos de datos).
- [ ] Escala: Valores expresados en Millones de COP para legibilidad.
- [ ] Estética: Dark Mode con gradiente YlGnBu (Amarillo-Verde-Azul).

In [44]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    12:35:00
# Versión: v7.0 (Heatmap Gold Standard)
# Proyecto: CRSN Slides (3J / VitaNova)
# Propósito: Generación de Mapa de Calor Mensual 2015-2025 en 16:9 Dark
# -----------------------------------------------------------------------------

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from datetime import datetime

# CONFIGURACIÓN DE RUTAS
EXCEL_PATH = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx"
LOGO_DARK = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png"
OUTPUT_DIR = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/"

def generar_heatmap_v70(path, logo, out):
    # 1. Extracción de Matriz Mensual
    xl = pd.ExcelFile(path)
    years = [str(y) for y in range(2015, 2026)]
    months = ["ENE", "FEB", "MAR", "ABR", "MAY", "JUN", "JUL", "AGO", "SEP", "OCT", "NOV", "DIC"]

    matrix = []
    for yr in years:
        df = pd.read_excel(xl, sheet_name=yr, header=None)
        row = df[df[0].astype(str).str.strip().str.upper() == "DIEZMOS"]
        if not row.empty:
            vals = pd.to_numeric(row.iloc[0, 1:13], errors='coerce').fillna(0).tolist()
            matrix.append(vals)
        else:
            matrix.append([0]*12)

    df_hm = pd.DataFrame(matrix, index=years, columns=months)

    # 2. Nomenclatura PJLA+
    pjla_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    pptx_name = f"{pjla_id}_DIEZMOS_HEATMAP_v7.0.pptx"
    df_hm.to_csv(os.path.join(out, f"{pjla_id}_DATA_HEATMAP_v7.0.csv"))

    # 3. Plot Estilizado
    plt.figure(figsize=(14, 7), facecolor='#0A0A14')
    sns.set(style="white")
    ax = sns.heatmap(df_hm / 1e6, annot=True, fmt=".0f", cmap="YlGnBu",
                     linewidths=.5, cbar_kws={'label': 'Millones COP'},
                     annot_kws={"size": 10, "weight": "bold"})

    plt.title("Intensidad de Recaudación Mensual (M$)", color='white', fontsize=16, pad=20)
    plt.xticks(color='white', fontweight='bold')
    plt.yticks(color='white', fontweight='bold')

    img_tmp = f"hm_{pjla_id}.png"
    plt.savefig(img_tmp, dpi=300, bbox_inches='tight', facecolor='#0A0A14')
    plt.close()

    # 4. Construcción Slide 16:9
    prs = Presentation()
    prs.slide_width, prs.slide_height = Inches(13.33), Inches(7.5)
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    slide.background.fill.solid()
    slide.background.fill.fore_color.rgb = RGBColor(10, 10, 20)

    if os.path.exists(logo):
        slide.shapes.add_picture(logo, Inches(0.5), Inches(0.4), height=Inches(1.2))

    t_box = slide.shapes.add_textbox(Inches(2.4), Inches(0.4), Inches(10), Inches(1))
    p = t_box.text_frame.paragraphs[0]
    p.text = "DIEZMOS: MAPA DE CALOR MENSUAL (2015-2025)"
    p.font.bold, p.font.size, p.font.color.rgb = True, Pt(44), RGBColor(255, 255, 255)

    slide.shapes.add_picture(img_tmp, Inches(0.5), Inches(1.8), width=Inches(12.33))

    save_path = os.path.join(out, pptx_name)
    prs.save(save_path)
    os.remove(img_tmp)
    return save_path

# Ejecución
try:
    path_final = generar_heatmap_v70(EXCEL_PATH, LOGO_DARK, OUTPUT_DIR)
    print(f"✅ Heatmap generado y blindado: {path_final}")
    from google.colab import files
    files.download(path_final)
except Exception as e:
    print(f"❌ Error en v7.0: {e}")

✅ Heatmap generado y blindado: /content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/20260129_172145_DIEZMOS_HEATMAP_v7.0.pptx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [1]:
# -----------------------------------------------------------------------------
# Autor:   Pedro J. Lancheros (Peter Boat)
# Fecha:   2026-01-29
# Hora:    17:34
# Versión: v8.0 (Hardcore Reset - Único)
# Propósito: Inicialización de Rutas y Nomenclatura Blindada
# -----------------------------------------------------------------------------
import os
from datetime import datetime

# CONFIGURACIÓN MAESTRA
EXCEL_MASTER = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx"
LOGO_DARK = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/crsn-logo-dark-circulo.png"
OUTPUT_DIR = "/content/drive/MyDrive/01_projects/crsn-slides-actividades-2025/"

# Generador de ID Único (Segundos SS)
def get_pjla_id():
    return datetime.now().strftime("%Y%m%d_%H%M%S")

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

print(f"🚀 [SISTEMA] Entorno v8.0 inicializado. ID de sesión: {get_pjla_id()}")

🚀 [SISTEMA] Entorno v8.0 inicializado. ID de sesión: 20260129_222853


## BLOQUE 1 — Pipeline Único: Auditoría, Barras 400 & Heatmap (v8.5)

**Lead Data Scientist:** Pedro J. Lancheros  
**Proyecto:** CRSN Slides (3J / VitaNova)  
**Versión:** v8.5 (Código Unificado)  
**Sello:** 2026-01-29 [jue] 17:52  

### 🎯 Propósito Técnico
Ejecutar en un solo flujo la extracción de datos financieros, la validación del crecimiento (Índice 400), la generación de backups en CSV y la creación de dos archivos PPTX independientes (Crecimiento e Intensidad) bajo el estándar 16:9 Dark Navy.

### 📥 Entradas Requeridas
- Excel Maestro: `vf.ESTADISTICAS_SABANA_NORTE_2015-2025.xlsx`
- Logo Institucional: `crsn-logo-dark-circulo.png`

### 📤 Salidas Generadas
- CSV: `[ID]_AUDITORIA_DETALLADA_v8.5.csv`
- PPTX 1: `[ID]_DIEZMOS_BARRAS_v8.5.pptx`
- PPTX 2: `[ID]_DIEZMOS_HEATMAP_v8.5.pptx`

### 🛡️ Validaciones Críticas
- [ ] Verificación de sumatoria mensual (Ene-Dic).
- [ ] Ratio 2025/2015 = 4.00x.
- [ ] Estética: Barras al 95% y Heatmap con anotaciones en Millones (M$).

**Estado:** ✅ Listo para ejecución única